In [1]:
import os
import cv2
import itertools

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

READ IMAGE & CONVERT INTO GRAYSCALE

In [2]:
def detect_face(img):
    img = img[70:195,78:172]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (50, 50))
    return img

In [3]:
def print_progress(val, val_len, folder, bar_size=20):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] (%d samples)\t label : %s \t\t" % (progr, val+1, folder), end="\r")

In [4]:
dataset_folder = "dataset/"

names = []
images = []
for folder in os.listdir(dataset_folder):
    files = os.listdir(os.path.join(dataset_folder, folder))[:150]
    if len(files) < 50 :
        continue
    for i, name in enumerate(files): 
        if name.find(".jpg") > -1 :
            img = cv2.imread(os.path.join(dataset_folder + folder, name))
            img = detect_face(img) # detect face using mtcnn and crop to 100x100
            if img is not None :
                images.append(img)
                names.append(folder)

                print_progress(i, len(files), folder)


[####################] (77 samples)	 label : Ariel_Sharon 		
[####################] (60 samples)	 label : Junichiro_Koizumi 		


In [5]:
print("number of samples :", len(names))

number of samples : 221


ENCODING LABEL & CATEGORICALIZATION

In [6]:
le = LabelEncoder()

le.fit(names)

labels = le.classes_

name_vec = le.transform(names)

categorical_name_vec = to_categorical(name_vec)

In [7]:
print("number of class :", len(labels))
print(labels)

number of class : 3
['Ariel_Sharon' 'Junichiro_Koizumi' 'Phillia']


In [8]:
print(name_vec)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [9]:
print(categorical_name_vec)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

LOAD MODEL & IMPLEMENT FACE RECOGNITION

In [10]:
from keras.models import load_model

In [11]:
def draw_ped(img, label, x0, y0, xt, yt, color=(255,127,0), text_color=(255,255,255)):

    (w, h), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    cv2.rectangle(img,
                  (x0, y0 + baseline),  
                  (max(xt, x0 + w), yt), 
                  color, 
                  2)
    cv2.rectangle(img,
                  (x0, y0 - h),  
                  (x0 + w, y0 + baseline), 
                  color, 
                  -1)  
    cv2.putText(img, 
                label, 
                (x0, y0),                   
                cv2.FONT_HERSHEY_SIMPLEX,     
                0.5,                          
                text_color,                
                1,
                cv2.LINE_AA) 
    return img

In [13]:
# --------- load Haar Cascade model -------------
face_cascade = cv2.CascadeClassifier('D:\Project_Face_Recognition 2\haarcascade_frontalface_default.xml')

# --------- load Keras CNN model -------------
model = load_model("D:\Project_Face_Recognition 2\model-philli.h5")
print("[INFO] finish load model...")

cap = cv2.VideoCapture(0)
while cap.isOpened() :
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        for (x, y, w, h) in faces:
            
            face_img = gray[y:y+h, x:x+w]
            face_img = cv2.resize(face_img, (50, 50))
            face_img = face_img.reshape(1, 50, 50, 1)
            
            result = model.predict(face_img)
            idx = result.argmax(axis=1)
            confidence = result.max(axis=1)*100
            if confidence > 80:
                label_text = "%s (%.2f %%)" % (labels[idx], confidence)
            else :
                label_text = "N/A"
            frame = draw_ped(frame, label_text, x, y, x + w, y + h, color=(0,255,255), text_color=(50,50,50))
       
        cv2.imshow('Detect Face', frame)
    else :
        break
    if cv2.waitKey(10) == ord('q'):
        break
        
cv2.destroyAllWindows()
cap.release()

[INFO] finish load model...
1/1 [==============================] - ETA: 0s

C:\Users\W10\AppData\Local\Temp\ipykernel_1344\486231864.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label_text = "%s (%.2f %%)" % (labels[idx], confidence)


1/1 [==============================] - 0s 28ms/step


TESTING FILE PICTURE ONE BY ONE

In [15]:
import cv2
from keras.models import load_model

# load Haar Cascade model
face_cascade = cv2.CascadeClassifier('D:\\Project_Face_Recognition 2\\haarcascade_frontalface_default.xml')

# load Keras CNN model
model = load_model("D:\\Project_Face_Recognition 2\\model-philli.h5")
print("[INFO] finish load model...")

# Function to draw text and bounding box around detected face
def draw_ped(frame, text, x, y, x_plus_w, y_plus_h, color, text_color):
    cv2.rectangle(frame, (x, y), (x_plus_w, y_plus_h), color, 2)
    cv2.putText(frame, text, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    return frame

# Load image for testing
image_path = "D:\\Project_Face_Recognition 2\\Raffi-Ahmad-2020.jpg"
image = cv2.imread(image_path)

# Check if image is loaded successfully
if image is None:
    print("Error: Image not loaded.")
else:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)

    # Process each detected face
    for (x, y, w, h) in faces:
        face_img = gray[y:y+h, x:x+w]
        face_img = cv2.resize(face_img, (50, 50))
        face_img = face_img.reshape(1, 50, 50, 1)

        # Predict using the loaded model
        result = model.predict(face_img)
        idx = result.argmax(axis=1)
        confidence = result.max(axis=1) * 100
        if confidence > 80:
            label_text = "%s (%.2f %%)" % (labels[idx], confidence)
        else:
            label_text = "N/A"
        image = draw_ped(image, label_text, x, y, x + w, y + h, color=(0, 255, 255), text_color=(50, 50, 50))

    # Display the image with detected faces
    cv2.imshow('Detect Face', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


[INFO] finish load model...
1/1 [==============================] - 0s 131ms/step


C:\Users\W10\AppData\Local\Temp\ipykernel_1344\136357329.py:41: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label_text = "%s (%.2f %%)" % (labels[idx], confidence)


TESTING PICTURE ALL OF THEM BY VISUAL

In [ ]:
# import cv2
# from keras.models import load_model
# import os

# # Load Haar Cascade model
# face_cascade = cv2.CascadeClassifier('D:\\Project_Face_Recognition\\haarcascade_frontalface_default.xml')

# # Load Keras CNN model
# model = load_model("D:\\Project_Face_Recognition 2\\model-philli.h5")
# print("[INFO] finish load model...")

# # Function to draw text and bounding box around detected face
# def draw_ped(frame, text, x, y, x_plus_w, y_plus_h, color, text_color):
#     cv2.rectangle(frame, (x, y), (x_plus_w, y_plus_h), color, 2)
#     cv2.putText(frame, text, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
#     return frame

# # Folder path containing images
# folder_path = "D:\\Project_Face_Recognition 2\\dataset\\Junichiro_Koizumi"

# # Iterate over all files in the folder
# for filename in os.listdir(folder_path):
#     if filename.endswith(".jpg") or filename.endswith(".png"):  # Check if file is an image file
#         # Load image
#         image_path = os.path.join(folder_path, filename)
#         image = cv2.imread(image_path)

#         # Check if image is loaded successfully
#         if image is None:
#             print("Error: Image not loaded.")
#             continue
#         else:
#             gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#             # Detect faces in the image
#             faces = face_cascade.detectMultiScale(gray, 1.1, 5)

#             # Process each detected face
#             for (x, y, w, h) in faces:
#                 face_img = gray[y:y+h, x:x+w]
#                 face_img = cv2.resize(face_img, (50, 50))
#                 face_img = face_img.reshape(1, 50, 50, 1)

#                 # Predict using the loaded model
#                 result = model.predict(face_img)
#                 idx = result.argmax(axis=1)
#                 confidence = result.max(axis=1) * 100
#                 if confidence > 80:
#                     label_text = "Phillia (%.2f %%)" % confidence
#                 else:
#                     label_text = "Unknown"
#                 image = draw_ped(image, label_text, x, y, x + w, y + h, color=(0, 255, 255), text_color=(50, 50, 50))

#             # Display the image with detected faces
#             cv2.imshow('Detect Face', image)
#             cv2.waitKey(0)

# # Close OpenCV windows
# cv2.destroyAllWindows()


In [16]:
import cv2
from keras.models import load_model
import os

# Load Haar Cascade model
face_cascade = cv2.CascadeClassifier('D:\\Project_Face_Recognition 2\\haarcascade_frontalface_default.xml')

# Load Keras CNN model
model = load_model("D:\\Project_Face_Recognition 2\\model-philli.h5")
print("[INFO] finish load model...")

# Function to draw text and bounding box around detected face
def draw_ped(frame, text, x, y, x_plus_w, y_plus_h, color, text_color):
    cv2.rectangle(frame, (x, y), (x_plus_w, y_plus_h), color, 2)
    cv2.putText(frame, text, (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    return frame

# Folder path containing images
folder_path = "D:\\Project_Face_Recognition 2\\dataset\\Ariel_Sharon"
folder_name = os.path.basename(folder_path)  # Extracting the folder name

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Check if file is an image file
        # Load image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)

        # Check if image is loaded successfully
        if image is None:
            print("Error: Image not loaded.")
            continue
        else:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Detect faces in the image
            faces = face_cascade.detectMultiScale(gray, 1.1, 5)

            # Process each detected face
            for (x, y, w, h) in faces:
                face_img = gray[y:y+h, x:x+w]
                face_img = cv2.resize(face_img, (50, 50))
                face_img = face_img.reshape(1, 50, 50, 1)

                # Predict using the loaded model
                result = model.predict(face_img)
                idx = result.argmax(axis=1)
                confidence = result.max(axis=1) * 100
                if confidence > 80:
                    label_text = "{} ({:.2f} %)".format(folder_name, confidence[0])
                else:
                    label_text = "Unknown"
                image = draw_ped(image, label_text, x, y, x + w, y + h, color=(0, 255, 255), text_color=(50, 50, 50))

            # Display the image with detected faces
            cv2.imshow('Detect Face', image)
            cv2.waitKey(0)

# Close OpenCV windows
cv2.destroyAllWindows()


[INFO] finish load model...
1/1 [==============================] - 0s 33ms/step


TESTING PICTURE ALL OF THEM BY LIST DATA 

In [17]:
import cv2
from keras.models import load_model
import os

# Load Haar Cascade model
face_cascade = cv2.CascadeClassifier('D:\\Project_Face_Recognition 2\\haarcascade_frontalface_default.xml')

# Load Keras CNN model
model = load_model("D:\\Project_Face_Recognition 2\\model-philli.h5")
print("[INFO] finish load model...")

# Function to calculate confidence score for a face
def calculate_confidence(face_img):
    face_img = cv2.resize(face_img, (50, 50))
    face_img = face_img.reshape(1, 50, 50, 1)

    # Predict using the loaded model
    result = model.predict(face_img)
    confidence = result.max(axis=1) * 100
    return confidence[0]

# Folder path containing images
folder_path = "D:\\Project_Face_Recognition 2\\dataset\\Ariel_Sharon"

# Iterate over all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Check if file is an image file
        # Load image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)

        # Check if image is loaded successfully
        if image is None:
            print("Error: Image not loaded.")
            continue
        else:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

            # Detect faces in the image
            faces = face_cascade.detectMultiScale(gray, 1.1, 5)

            # Process each detected face
            for (x, y, w, h) in faces:
                face_img = gray[y:y+h, x:x+w]
                confidence = calculate_confidence(face_img)
                print("File:", filename, "- Confidence:", confidence)

# Close OpenCV windows
cv2.destroyAllWindows()


[INFO] finish load model...
1/1 [==============================] - 0s 220ms/step
File: Ariel_Sharon_0001.jpg - Confidence: 99.99987
1/1 [==============================] - 0s 42ms/step
File: Ariel_Sharon_0002.jpg - Confidence: 100.0
1/1 [==============================] - 0s 33ms/step
File: Ariel_Sharon_0003.jpg - Confidence: 99.99996
1/1 [==============================] - 0s 37ms/step
File: Ariel_Sharon_0004.jpg - Confidence: 95.75351
1/1 [==============================] - 0s 46ms/step
File: Ariel_Sharon_0004.jpg - Confidence: 98.34436
1/1 [==============================] - 0s 52ms/step
File: Ariel_Sharon_0005.jpg - Confidence: 100.0
1/1 [==============================] - 0s 31ms/step
File: Ariel_Sharon_0006.jpg - Confidence: 99.986145
1/1 [==============================] - 0s 40ms/step
File: Ariel_Sharon_0007.jpg - Confidence: 74.553154
1/1 [==============================] - 0s 33ms/step
File: Ariel_Sharon_0008.jpg - Confidence: 100.0
1/1 [==============================] - 0s 44ms/step